In [1]:
import sys
sys.path.append('../')

In [2]:
from utils.utils import Utils
from utils.preprocess import Preprocess

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/eastwind/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eastwind/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/eastwind/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eastwind/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/eastwind/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /home/eastwind/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [4]:
%%time
utils = Utils()
pre = Preprocess(mode="normalize")

CPU times: user 43.7 s, sys: 5.67 s, total: 49.4 s
Wall time: 1min 45s


In [19]:
%%time
data = utils.read_data("../dataset/train/train-empathy-distress-prediction-task-normalized.csv")
data

CPU times: user 26 ms, sys: 110 µs, total: 26.1 ms
Wall time: 25.5 ms


,message_id,response_id,article_id,empathy,distress,empathy_bin,distress_bin,essay,emotion,gender,...,income,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,R_1hGrPtWM4SumG0U_1,R_1hGrPtWM4SumG0U,67,5.667,4.375,1,1,it is really disheartening to read about thes...,sadness,1,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
1,R_1hGrPtWM4SumG0U_2,R_1hGrPtWM4SumG0U,86,4.833,4.875,1,1,the phone lines from the suicide prevention l...,sadness,1,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
2,R_1hGrPtWM4SumG0U_3,R_1hGrPtWM4SumG0U,206,5.333,3.500,1,0,"no matter what your heritage , you should be ...",neutral,1,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
3,R_1hGrPtWM4SumG0U_4,R_1hGrPtWM4SumG0U,290,4.167,5.250,1,1,it is frightening to learn about all these sh...,fear,1,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
4,R_1hGrPtWM4SumG0U_5,R_1hGrPtWM4SumG0U,342,5.333,4.625,1,1,the eldest generation of russians are not bei...,sadness,1,...,50000,6.0,5.0,5.0,5.5,5.5,3.571,2.000,3.429,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,R_DHy2Rcz9Hym8jgl_1,R_DHy2Rcz9Hym8jgl,16,1.000,1.000,0,0,"These days , women are winning in sports . Th...",joy,2,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1856,R_DHy2Rcz9Hym8jgl_2,R_DHy2Rcz9Hym8jgl,158,1.000,1.000,0,0,I hate SIS . They are a group full of hate an...,anger,2,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1857,R_DHy2Rcz9Hym8jgl_3,R_DHy2Rcz9Hym8jgl,203,1.000,1.000,0,0,This is just disgusting . I cannot believe th...,disgust,2,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429
1858,R_DHy2Rcz9Hym8jgl_4,R_DHy2Rcz9Hym8jgl,253,1.000,1.000,0,0,I feel like the world is so corrupt that it n...,sadness,2,...,30000,7.0,7.0,7.0,7.0,7.0,5.000,1.286,4.286,4.429


In [13]:
post_no = 111
data.essay.values.tolist()[post_no]

'I feel like Venezuela is stuck. I truly feel like Maduro is not even a bad person just looked at wrong in public. I think Hugo Chavez was not all that bad neither. I do think they need a change from that communist style though. I think that the thing that keeps people up at night is that they need to change. That is how I feel about the whole Venezuela situation and it should change soon hopefully for their well being. Anything else might be too late or too much in the long run of things. We need to help make a change by donating to good organizations.'

In [14]:
%%time
normalized_data = pre.normalize_corpus(data.copy(), column_name="essay")
normalized_data

CPU times: user 8.19 s, sys: 57.5 ms, total: 8.25 s
Wall time: 9.07 s


,message_id,response_id,article_id,essay,gender,education,race,age,income,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,R_3QLVVnAgRBRH41U_1,R_3QLVVnAgRBRH41U,13,The story about the air strikes is very sadde...,1.0,4.0,3.0,20.0,24000.0,5.5,5.5,4.0,5.0,4.5,4.429,2.286,4.143,3.143
1,R_3QLVVnAgRBRH41U_2,R_3QLVVnAgRBRH41U,127,It is clear that climate change is something ...,1.0,4.0,3.0,20.0,24000.0,5.5,5.5,4.0,5.0,4.5,4.429,2.286,4.143,3.143
2,R_3QLVVnAgRBRH41U_3,R_3QLVVnAgRBRH41U,188,I did not know this comedian but thinking abo...,1.0,4.0,3.0,20.0,24000.0,5.5,5.5,4.0,5.0,4.5,4.429,2.286,4.143,3.143
3,R_3QLVVnAgRBRH41U_4,R_3QLVVnAgRBRH41U,307,I am an affirmed believer that your punishmen...,1.0,4.0,3.0,20.0,24000.0,5.5,5.5,4.0,5.0,4.5,4.429,2.286,4.143,3.143
4,R_3QLVVnAgRBRH41U_5,R_3QLVVnAgRBRH41U,409,"Okay , I hate hearing about disease outbreaks...",1.0,4.0,3.0,20.0,24000.0,5.5,5.5,4.0,5.0,4.5,4.429,2.286,4.143,3.143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,R_r3CoQ7YjIwxVvJT_1,R_r3CoQ7YjIwxVvJT,2,"While I feel bad for the arrest charges , I d...",2.0,2.0,1.0,37.0,93000.0,4.0,6.0,1.0,5.0,2.5,3.857,4.143,4.286,3.286
266,R_r3CoQ7YjIwxVvJT_2,R_r3CoQ7YjIwxVvJT,131,This is such a tragedy . All I can think abou...,2.0,2.0,1.0,37.0,93000.0,4.0,6.0,1.0,5.0,2.5,3.857,4.143,4.286,3.286
267,R_r3CoQ7YjIwxVvJT_3,R_r3CoQ7YjIwxVvJT,230,I am still on the fence regarding climate cha...,2.0,2.0,1.0,37.0,93000.0,4.0,6.0,1.0,5.0,2.5,3.857,4.143,4.286,3.286
268,R_r3CoQ7YjIwxVvJT_4,R_r3CoQ7YjIwxVvJT,326,I can understand why the Jewish community wou...,2.0,2.0,1.0,37.0,93000.0,4.0,6.0,1.0,5.0,2.5,3.857,4.143,4.286,3.286


In [15]:
normalized_data.essay.values.tolist()[post_no]

' I feel like Venezuela is stuck . I truly feel like Madero is not even a bad person just looked at wrong in public . I think Hugo Chaver was not all that bad neither . I do think they need a change from that communist style though . I think that the thing that keeps people up at night is that they need to change . That is how I feel about the whole Venezuela situation and it should change soon hopefully for their well being . Anything else might be too late or too much in the long run of things . We need to help make a change by donating to good organizations .'

In [16]:
normalized_data.to_csv("../dataset/dev-empathy-distress-prediction-task-normalized.csv", index=False)